Heroes of Pymoli Data Analysis
1. A majority of the purchases are made by people under 30. People between the ages of 20 - 24 make the most purchases however they do not make the most expensive purchases
2. The most popular items are below the average purchase price
3. The most profitable items are priced higher than the average price while the most popular are priced below the average.


In [12]:
#Import Dependencies
import pandas as pd

In [13]:
# Create a path to the json and  read it into a Pandas DataFrame
json_path = "HeroesOfPymoli/purchase_data.json"
heroes_df = pd.read_json(json_path)
heroes_df.head()


,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [14]:
#Find Player Count
player_count = heroes_df["SN"].nunique()
player_count
player_count_df = pd.DataFrame({'Total Players' : [player_count]})
player_count_df





,Total Players
0,573


In [15]:
#Number of Unique Items
unique_items = heroes_df["Item Name"].nunique()
#Average Purchase Price
average_price = heroes_df["Price"].mean()
#Total Number of Purchases
Total_purchases = heroes_df["Price"].count()
#Total Revenue
Total_Revenue = heroes_df["Price"].sum()

purchase_analysis_df = pd.DataFrame({'Unique Items' : [unique_items],
                                     'Average Purchase Price' : [average_price],
                                     'Total Purchases': [Total_purchases],
                                     'Total Revenue' : [Total_Revenue]})
purchase_analysis_df['Average Purchase Price'] = purchase_analysis_df['Average Purchase Price'].map("${:.2f}".format)
purchase_analysis_df['Total Revenue'] = purchase_analysis_df['Total Revenue'].map("${:,.2f}".format)

purchase_analysis_df



,Average Purchase Price,Total Purchases,Total Revenue,Unique Items
0,$2.93,780,"$2,286.33",179


In [16]:
#Percentage and Count Per Gender
male_players = heroes_df.loc[heroes_df["Gender"] == "Male"]
male_count = male_players["SN"].nunique()
male_percent = (male_count/player_count)*100
female_players = heroes_df.loc[heroes_df["Gender"] == "Female"]
female_count = female_players["SN"].nunique()
female_percent = (female_count/player_count)*100
other_players = heroes_df.loc[~heroes_df["Gender"].isin(["Male","Female"])]
other_count = other_players["SN"].nunique()
other_percent = (other_count/player_count)*100

gender_df = pd.DataFrame({'Gender' : ["Male", "Female", "Other/Non-disclosed"],
                          'Percentage of Players' : [male_percent, female_percent, other_percent],
                          'Total Count' : [male_count, female_count, other_count]})
gender_df = gender_df.set_index('Gender')
gender_df['Percentage of Players'] = gender_df['Percentage of Players'].map("{:.2f}%".format)
gender_df['Total Count'] = gender_df['Total Count'].map("{:}".format)

gender_df
                                                

,Percentage of Players,Total Count
Gender,,
Male,81.15%,465
Female,17.45%,100
Other/Non-disclosed,1.40%,8


In [17]:
#purchase analysis by gender
male_purchase = male_players["Price"].count()
male_total_value = male_players["Price"].sum()
male_average = male_players["Price"].mean()
male_norm = male_total_value/male_count
female_purchase = female_players["Price"].count()
female_total_value = female_players["Price"].sum()
female_average = female_players["Price"].mean()
female_norm = female_total_value/female_count
other_purchase = other_players["Price"].count()
other_total_value = other_players["Price"].sum()
other_average = other_players["Price"].mean()
other_norm = other_total_value/other_count

gender_purchase_df = pd.DataFrame({"Gender" : ["Male", "Female", "Other/Non-Disclosed"],
                                  "Purchase Count" : [male_purchase, female_purchase, other_purchase],
                                  "Average Purchase Price" : [male_average, female_average, other_average],
                                  "Total Purchase Price" : [male_total_value, female_total_value, other_total_value],
                                  "Normalized Totals" :[male_norm, female_norm, other_norm]})
gender_purchase_df = gender_purchase_df.set_index("Gender")
gender_purchase_df['Total Purchase Price'] = gender_purchase_df['Total Purchase Price'].map("${:,.2f}".format)
gender_purchase_df['Average Purchase Price'] = gender_purchase_df['Average Purchase Price'].map("${:.2f}".format)
gender_purchase_df['Normalized Totals'] = gender_purchase_df['Normalized Totals'].map("${:.2f}".format)
gender_purchase_df


,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Price
Gender,,,,
Male,$2.95,$4.02,633,"$1,867.68"
Female,$2.82,$3.83,136,$382.91
Other/Non-Disclosed,$3.25,$4.47,11,$35.74


In [18]:
#age demographics
#print(heroes_df["Age"].max())
#print(heroes_df["Age"].min())

#Create Bins
bins = [0, 10, 15, 20, 25, 30, 35, 40, 150]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

#new dataframe and groupby
age_summary_df = heroes_df.copy()
age_summary_df["Age_Range"] = pd.cut(age_summary_df["Age"], bins, labels=group_names)
age_groups = age_summary_df.groupby(["Age_Range"])

age_count = age_groups["SN"].count()
age_percentage = (age_count/player_count)*100


age_df = pd.DataFrame({"Total Count" : age_count,
                      "Percentage of Players" : age_percentage})
age_df['Percentage of Players'] = age_df['Percentage of Players'].map("{:.2f}%".format)

age_df


,Percentage of Players,Total Count
Age_Range,,
<10,5.58%,32
10-14,13.61%,78
15-19,32.11%,184
20-24,53.23%,305
25-29,13.26%,76
30-34,10.12%,58
35-39,7.68%,44
40+,0.52%,3


In [19]:
#Age Analysis
unique_age_group = age_groups['SN'].nunique()
age_purchase = age_groups["Price"].count()
age_average = age_groups["Price"].mean()
age_total = age_groups["Price"].sum()
age_norm = age_total/unique_age_group

age_analysis_df = pd.DataFrame({"Purchase Count" : age_purchase, 
                             "Average Purchase Price" : age_average, 
                             "Total Purchase Price" : age_total, 
                             "Normalized Totals" : age_norm})

age_analysis_df['Total Purchase Price'] = age_analysis_df['Total Purchase Price'].map("${:,.2f}".format)
age_analysis_df['Average Purchase Price'] = age_analysis_df['Average Purchase Price'].map("${:.2f}".format)
age_analysis_df['Normalized Totals'] = age_analysis_df['Normalized Totals'].map("${:.2f}".format)

age_analysis_df


,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Price
Age_Range,,,,
<10,$3.02,$4.39,32,$96.62
10-14,$2.87,$4.15,78,$224.15
15-19,$2.87,$3.80,184,$528.74
20-24,$2.96,$3.86,305,$902.61
25-29,$2.89,$4.23,76,$219.82
30-34,$3.07,$4.05,58,$178.26
35-39,$2.90,$5.10,44,$127.49
40+,$2.88,$2.88,3,$8.64


In [20]:
#Top Spenders
SN_group = heroes_df.groupby(["SN"])
player_purchase_count = SN_group["Price"].count()
player_purchase_total = SN_group["Price"].sum()
player_purchase_average = SN_group["Price"].mean()

top_spender_df = pd.DataFrame({"Purchase Count" : player_purchase_count,
                              "Average Purchase Price" : player_purchase_average,
                              "Total Purchase Value" : player_purchase_total})

top_spender_df.sort_values("Total Purchase Value", ascending=False, inplace=True)

top_spender_df["Total Purchase Value"] = top_spender_df["Total Purchase Value"].map("${:.2f}".format)
top_spender_df["Average Purchase Price"] = top_spender_df["Average Purchase Price"].map("${:.2f}".format)

top_spender_df.head()




,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Undirrala66,$3.41,5,$17.06
Saedue76,$3.39,4,$13.56
Mindimnya67,$3.18,4,$12.74
Haellysu29,$4.24,3,$12.73
Eoda93,$3.86,3,$11.58


In [21]:
# Most popular item

#Create new dataframe for most popular item
popular_group = heroes_df.groupby(["Item ID"])
purchase_count = popular_group["Price"].count()
purchase_total = popular_group["Price"].sum()
top_item_df = pd.DataFrame({"Purchase Count" : purchase_count,
                            "Total Purchase Value" : purchase_total})

#Remove duplicates from OG dataframe and keep only columns needed
drop_dups_df = heroes_df.drop_duplicates(["Item ID"], keep = "last")
item_id_index_df = drop_dups_df.set_index('Item ID')
item_id_index_df = item_id_index_df[["Item Name", "Price"]]

#combine two dataframes to get all the information needed
frames = [top_item_df, item_id_index_df]
top_item_df = pd.concat(frames, axis=1)
top_item_df.sort_values("Purchase Count", ascending=False, inplace=True)

#format columns
top_item_df["Total Purchase Value"] = top_item_df["Total Purchase Value"].map("${:.2f}".format)
top_item_df["Price"] = top_item_df["Price"].map("${:.2f}".format)
top_item_df = top_item_df[["Item Name", "Purchase Count", "Price", "Total Purchase Value"]]


top_item_df.head()


,Item Name,Purchase Count,Price,Total Purchase Value
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [22]:
#Most Profitable Items

#Create new dataframe for most popular item
profit_group = heroes_df.groupby(["Item ID"])
profit_purchase_count = profit_group["Price"].count()
profit_purchase_total = profit_group["Price"].sum()
profitable_item_df = pd.DataFrame({"Purchase Count" : purchase_count,
                            "Total Purchase Value" : purchase_total})


frames = [profitable_item_df, item_id_index_df]
profitable_item_df = pd.concat(frames, axis=1)
profitable_item_df.sort_values("Total Purchase Value", ascending=False, inplace=True)

profitable_item_df["Total Purchase Value"] = profitable_item_df["Total Purchase Value"].map("${:.2f}".format)
profitable_item_df["Price"] = profitable_item_df["Price"].map("${:.2f}".format)
profitable_item_df = profitable_item_df[["Item Name", "Purchase Count", "Price", "Total Purchase Value"]]

profitable_item_df.head()


,Item Name,Purchase Count,Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
